In [1]:
data_var = '2023-12-28'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7496,2023-12-28,Eua Ncaa,23:00,Oregon,USC,1.69,2.22,150.5,1.91,1.91,-3.5,2.00,1.71,O8KHtbCB,0.591716,0.450450,0.523560,0.523560,0.042166,105.724,31.108694,0.294244,2.4,1.341641,0.559017,115.08,1.382,0.303760,0.219797,40.0,152.996,93.279371,0.609685,1.2,1.643168,1.369306,83.74,1.924,1.321374,0.686785,14.0,84,79,1.37,1.06,105.330,0.000,0.191696,0.000000,0.110545,1.40,0.280,2.464286,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7497,2023-12-28,Eua Ncaa,23:00,Idaho State,Montana State,1.80,2.05,134.5,1.91,1.91,-2.5,2.00,1.71,WUFSP0oE,0.555556,0.487805,0.523560,0.523560,0.043360,198.370,68.814098,0.346898,0.6,1.341641,2.236068,198.36,3.032,1.164633,0.384114,-48.0,281.842,288.230441,1.022667,1.2,1.643168,1.369306,212.10,4.284,4.908796,1.145844,-40.0,57,70,3.48,3.03,0.000,0.000,0.091832,0.000000,0.110545,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7498,2023-12-28,Eua Ncaa,23:00,Idaho,Sacramento State,1.67,2.28,139.5,1.91,1.91,-4.5,2.05,1.71,pYJWOK0K,0.598802,0.438596,0.523560,0.523560,0.037399,344.342,345.968838,1.004724,1.8,1.643168,0.912871,220.43,5.130,5.578647,1.087456,13.0,196.048,109.043735,0.556209,0.6,1.341641,2.236068,93.60,2.918,1.373106,0.470564,-40.0,67,52,3.29,1.80,0.000,434.558,0.218398,0.000000,0.127881,0.00,0.000,inf,0.000000,0.0,0.000000,-0.58,-0.116,-11.034483,0.000000,0.0,0.000000
7499,2023-12-28,Eua Ncaa,23:00,East. Washington,Portland State,1.67,2.28,150.5,1.80,1.95,-4.5,2.05,1.71,Sd9yOvGQ,0.598802,0.438596,0.555556,0.512821,0.037399,605.536,267.736267,0.442148,0.6,1.341641,2.236068,646.80,8.392,3.395927,0.404662,-65.0,161.262,119.780335,0.742769,1.2,1.643168,1.369306,115.05,2.452,2.066136,0.842633,-3.0,66,65,9.80,1.77,0.000,225.886,0.218398,0.056569,0.127881,0.00,0.000,inf,0.000000,0.0,0.000000,2.04,0.408,3.137255,0.000000,0.0,0.000000
7500,2023-12-28,Eua Ncaa,22:30,Tennessee State,Tenn. Martin,1.51,2.61,161.5,1.80,1.95,-5.5,2.00,1.71,2HtfLxNK,0.662252,0.383142,0.555556,0.512821,0.045393,326.710,287.748626,0.880746,0.6,1.341641,2.236068,793.50,5.042,4.437727,0.880152,-48.0,424.926,374.374147,0.881034,0.6,1.341641,2.236068,272.09,5.706,5.895789,1.033261,-29.0,69,91,11.50,2.99,0.000,424.926,0.377581,0.056569,0.110545,0.00,0.000,inf,0.000000,0.0,0.000000,-3.37,-0.674,-2.388724,0.000000,0.0,0.000000
7501,2023-12-28,Eua Ncaa,22:30,Tennessee Tech,AR Little Rock,2.00,1.80,152.5,1.80,1.95,1.5,1.83,1.83,t8ubKdxR,0.500000,0.555556,0.555556,0.512821,0.055556,240.402,57.193126,0.237906,0.6,1.341641,2.236068,285.60,3.752,1.239786,0.330433,-48.0,122.634,25.960620,0.211692,1.2,1.643168,1.369306,103.80,1.662,0.314436,0.189191,-28.0,51,60,5.60,1.73,0.000,0.000,0.074432,0.056569,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7502,2023-12-28,Eua Ncaa,22:00,UTSA Roadrunners,Prairie View A&M,1.64,2.23,148.5,1.91,1.91,-4.5,2.05,1.71,2DCf1CES,0.609756,0.448430,0.523560,0.523560,0.058187,187.350,99.444893,0.530797,1.2,1.643168,1.369306,74.20,2.516,1.440653,0.572596,-7.0,395.144,167.050581,0.422759,1.2,1.643168,1.369306,274.96,5.908,2.534042,0.428917,-42.0,53,56,1.40,4.91,116.868,395.144,0.215604,0.000000,0.127881,-0.45,-0.090,-7.111111,0.000000,0.0,0.000000,2.63,0.526,2.338403,0.000000,0.0,0.000000
7503,2023-12-28,Eua Ncaa,21:00,Cleveland State,Oaklan

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
7496,23:00,Eua Ncaa,Oregon,USC,1.69,Back Home
7497,23:00,Eua Ncaa,Idaho State,Montana State,1.80,Back Home
7498,23:00,Eua Ncaa,Idaho,Sacramento State,1.67,Back Home
7499,23:00,Eua Ncaa,East. Washington,Portland State,1.67,Back Home
7501,22:30,Eua Ncaa,Tennessee Tech,AR Little Rock,2.00,Back Home
7502,22:00,Eua Ncaa,UTSA Roadrunners,Prairie View A&M,1.64,Back Home
7503,21:00,Eua Ncaa,Cleveland State,Oakland,1.73,Back Home
7508,22:00,Eua Nba,Chicago Bulls,Indiana Pacers,1.78,Back Home
7510,23:00,Eua Ncaa,UC Davis,UC Santa Barbara,2.14,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
7501,22:30,Eua Ncaa,Tennessee Tech,AR Little Rock,1.95,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7502,22:00,Eua Ncaa,UTSA Roadrunners,Prairie View A&M,1.64,Back Home
7503,21:00,Eua Ncaa,Cleveland State,Oakland,1.73,Back Home
